In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# --- 0. Load and de-duplicate columns ---
df_alite = pd.read_csv('alite_merge_before_processing.csv', low_memory=False)
df_manual = pd.read_csv('manual_merge_before_processing.csv', low_memory=False)

# Drop duplicate columns by case-insensitive name
norm_a = [c.lower().strip() for c in df_alite.columns]
df_alite = df_alite.loc[:, ~pd.Series(norm_a).duplicated().values]

norm_m = [c.lower().strip() for c in df_manual.columns]
df_manual = df_manual.loc[:, ~pd.Series(norm_m).duplicated().values]

# Identify keys
cc = next((c for c in df_alite.columns if 'country' in c.lower()), None)
yy = next((c for c in df_alite.columns if 'year' in c.lower()), None)

# --- 1. Summary Metrics ---
def compute_summary(df):
    rows, cols = df.shape
    avg_missing = df.isna().mean().mean() * 100
    unique_cy = df[[cc, yy]].drop_duplicates().shape[0] if cc and yy else np.nan
    return pd.Series({
        'Rows': rows,
        'Columns': cols,
        'AvgMissing (%)': round(avg_missing, 2),
        'UniqueCountryYear': unique_cy
    })

summary_df = pd.DataFrame({
    'ALITE_merge': compute_summary(df_alite),
    'Manual_merge': compute_summary(df_manual)
}).T

# --- 2. Country–Year Alignment (Jaccard) ---
if cc and yy:
    set_a = set(df_alite[[cc, yy]].drop_duplicates().apply(tuple, axis=1))
    set_m = set(df_manual[[cc, yy]].drop_duplicates().apply(tuple, axis=1))
    inter = len(set_a & set_m)
    uni = len(set_a | set_m)
    jacc = inter / uni if uni else np.nan
    align_df = pd.DataFrame([{
        'Shared Country-Years': inter,
        'Total Country-Years': uni,
        'Jaccard Index': round(jacc, 3)
    }])
else:
    align_df = pd.DataFrame()

# --- 3. Coverage of Unique Countries & Years ---
if cc and yy:
    countries_a = set(df_alite[cc].dropna())
    countries_m = set(df_manual[cc].dropna())
    years_a = set(df_alite[yy].dropna())
    years_m = set(df_manual[yy].dropna())
    coverage_df = pd.DataFrame({
        'ALITE_merge': [len(countries_a), len(years_a)],
        'Manual_merge': [len(countries_m), len(years_m)],
        'Common': [len(countries_a & countries_m), len(years_a & years_m)]
    }, index=['Unique Countries', 'Unique Years'])
else:
    coverage_df = pd.DataFrame()

# --- 4. Column Overlap & Listings ---
cols_a = set(df_alite.columns)
cols_m = set(df_manual.columns)
common = cols_a & cols_m
overlap_df = pd.DataFrame([{
    'ALITE_only': len(cols_a - common),
    'Common': len(common),
    'Manual_only': len(cols_m - common),
    'ALITE_only_cols': ', '.join(sorted(cols_a - common)),
    'Manual_only_cols': ', '.join(sorted(cols_m - common))
}])

# --- 5. Top-5 Missingness in Each ---
missing_a = (
    df_alite.isna().mean() * 100
).sort_values(ascending=False).head(5).round(2).reset_index()
missing_a.columns = ['Column', 'Missing (%)']

missing_m = (
    df_manual.isna().mean() * 100
).sort_values(ascending=False).head(5).round(2).reset_index()
missing_m.columns = ['Column', 'Missing (%)']

# --- 6. Data-Type Consistency Counts ---
dtype_records = []
for col in sorted(common):
    a_t = df_alite[col].dtype
    m_t = df_manual[col].dtype
    dtype_records.append({
        'Column': col,
        'ALITE_dtype': str(a_t),
        'Manual_dtype': str(m_t),
        'Consistent': a_t == m_t
    })
dtype_df = pd.DataFrame(dtype_records)
dtype_summary_df = pd.DataFrame([{
    'Consistent': int(dtype_df['Consistent'].sum()),
    'Inconsistent': int((~dtype_df['Consistent']).sum())
}])

# --- 7. Numeric Comparison (mean, MAD, KS-test) ---
numeric_common = [
    col for col in common
    if pd.api.types.is_numeric_dtype(df_alite[col])
    and pd.api.types.is_numeric_dtype(df_manual[col])
    and col not in {cc, yy}
]

if cc and yy:
    merged = pd.merge(
        df_alite[[cc, yy] + numeric_common],
        df_manual[[cc, yy] + numeric_common],
        on=[cc, yy],
        suffixes=('_alite', '_manual')
    )

stats = []
for col in numeric_common:
    a_vals = df_alite[col].dropna()
    m_vals = df_manual[col].dropna()
    if len(a_vals) > 1000:
        a_vals = a_vals.sample(1000, random_state=1)
    if len(m_vals) > 1000:
        m_vals = m_vals.sample(1000, random_state=1)
    if not a_vals.empty and not m_vals.empty:
        ks_s, p_v = ks_2samp(a_vals, m_vals)
        mad = np.nan
        if cc and yy:
            diff = merged[f'{col}_alite'] - merged[f'{col}_manual']
            mad = diff.abs().mean()
        stats.append({
            'Column': col,
            'ALITE_mean': round(a_vals.mean(), 3),
            'Manual_mean': round(m_vals.mean(), 3),
            'MeanAbsDiff': round(mad, 3),
            'KS_stat': round(ks_s, 3),
            'KS_pval': '{:.3e}'.format(p_v)
        })

stats_df = pd.DataFrame(stats).sort_values(by='MeanAbsDiff', ascending=False)

# --- 8. Generate Plots ---
plt.style.use('default')  # Use Matplotlib's default style

# Plot 1: Bar Plot - Summary Metrics Comparison
fig, ax = plt.subplots(figsize=(10, 6))
metrics = ['Rows', 'Columns', 'AvgMissing (%)', 'UniqueCountryYear']
x = np.arange(len(metrics))
width = 0.35

ax.bar(x - width/2, summary_df.loc['ALITE_merge', metrics], width, label='ALITE_merge', color='skyblue')
ax.bar(x + width/2, summary_df.loc['Manual_merge', metrics], width, label='Manual_merge', color='lightgreen')

ax.set_xticks(x)
ax.set_xticklabels(['Rows', 'Columns', 'Avg Missing (%)', 'Unique Country-Years'])
ax.set_ylabel('Value')
ax.set_title('Summary Metrics Comparison: ALITE_merge vs. Manual_merge')
ax.legend()
plt.tight_layout()
plt.savefig('summary_metrics_comparison.png')
plt.close()

# Plot 2: Bar Plot - Column Overlap
fig, ax = plt.subplots(figsize=(8, 6))
overlap_data = overlap_df[['ALITE_only', 'Common', 'Manual_only']].iloc[0]
ax.bar(overlap_data.index, overlap_data.values, color=['skyblue', 'gray', 'lightgreen'])
ax.set_ylabel('Number of Columns')
ax.set_title('Column Overlap: ALITE_merge vs. Manual_merge')
for i, v in enumerate(overlap_data.values):
    ax.text(i, v + 0.5, str(v), ha='center')
plt.tight_layout()
plt.savefig('column_overlap.png')
plt.close()

# Plot 3: Bar Plot - Top-5 Missingness Comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
ax1.barh(missing_a['Column'], missing_a['Missing (%)'], color='skyblue')
ax2.barh(missing_m['Column'], missing_m['Missing (%)'], color='lightgreen')
ax1.set_title('Top-5 Missingness: ALITE_merge')
ax2.set_title('Top-5 Missingness: Manual_merge')
ax1.set_xlabel('Missing (%)')
ax2.set_xlabel('Missing (%)')
ax1.set_ylabel('Column')
plt.tight_layout()
plt.savefig('top5_missingness_comparison.png')
plt.close()

# Plot 4: Box Plot - Numeric Column Mean Absolute Differences
fig, ax = plt.subplots(figsize=(10, 6))
ax.boxplot(stats_df['MeanAbsDiff'], vert=True, patch_artist=True, boxprops=dict(facecolor='lightgray'))
ax.scatter(range(1, len(stats_df) + 1), stats_df['MeanAbsDiff'], color='red', s=50, alpha=0.5)
ax.set_ylabel('Mean Absolute Difference (MAD)')
ax.set_title('Distribution of Mean Absolute Differences in Numeric Columns')
# Highlight significant outliers (MAD > 1e4)
outliers = stats_df[stats_df['MeanAbsDiff'] > 1e4]['Column']
for i, outlier in enumerate(outliers):
    mad_value = stats_df[stats_df['Column'] == outlier]['MeanAbsDiff'].iloc[0]
    ax.text(1.1, mad_value, outlier, color='red', va='center')
ax.set_xticks([])
plt.tight_layout()
plt.savefig('numeric_mad_boxplot.png')
plt.close()

# --- 9. Print Outputs and Plot Confirmation ---
print("\n=== Summary Metrics ===")
print(summary_df)

print("\n=== Country–Year Alignment ===")
print(align_df)

print("\n=== Coverage of Countries & Years ===")
print(coverage_df)

print("\n=== Column Overlap & Listings ===")
print(overlap_df.T)

print("\n=== Top-5 Missingness in ALITE_merge ===")
print(missing_a)

print("\n=== Top-5 Missingness in Manual_merge ===")
print(missing_m)

print("\n=== Data-Type Consistency Counts ===")
print(dtype_summary_df)

print("\n=== Numeric Columns Statistical Comparison ===")
print(stats_df.to_string(index=False))

print("\n=== Plots Generated ===")
print("Saved as: summary_metrics_comparison.png, column_overlap.png, top5_missingness_comparison.png, numeric_mad_boxplot.png")

/uufs/chpc.utah.edu/common/home/u1463636/.conda/envs/myProject/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = hypotest_fun_out(*samples, **kwds)



=== Summary Metrics ===
                 Rows  Columns  AvgMissing (%)  UniqueCountryYear
ALITE_merge   16080.0     77.0           20.30            11041.0
Manual_merge  11040.0     60.0           18.29            11040.0

=== Country–Year Alignment ===
   Shared Country-Years  Total Country-Years  Jaccard Index
0                 11040                11041            1.0

=== Coverage of Countries & Years ===
                  ALITE_merge  Manual_merge  Common
Unique Countries          184           184     184
Unique Years               60            60      60

=== Column Overlap & Listings ===
                                                                  0
ALITE_only                                                       21
Common                                                           56
Manual_only                                                       4
ALITE_only_cols   Labor force, total, Labor force, total_64, Une...
Manual_only_cols  Labor force, total_x, Labor force, to

In [11]:
import pandas as pd
import numpy as np
import os
from scipy.stats import ks_2samp

# — 0. Load and de-duplicate columns —
df_alite  = pd.read_csv('alite_merge.csv',  low_memory=False)
df_manual = pd.read_csv('manual_merge.csv', low_memory=False)

# drop duplicate columns by case‐insensitive name
norm_a = [c.lower().strip() for c in df_alite.columns]
df_alite  = df_alite.loc[:, ~pd.Series(norm_a).duplicated().values]

norm_m = [c.lower().strip() for c in df_manual.columns]
df_manual = df_manual.loc[:, ~pd.Series(norm_m).duplicated().values]

# identify keys
cc = next((c for c in df_alite.columns if 'country' in c.lower()), None)
yy = next((c for c in df_alite.columns if 'year'    in c.lower()), None)

# — 1. Summary Metrics —
def compute_summary(df):
    rows, cols   = df.shape
    avg_missing  = df.isna().mean().mean() * 100
    unique_cy    = df[[cc, yy]].drop_duplicates().shape[0] if cc and yy else np.nan
    return pd.Series({
        'Rows': rows,
        'Columns': cols,
        'AvgMissing (%)': round(avg_missing, 2),
        'UniqueCountryYear': unique_cy
    })

summary_df = pd.DataFrame({
    'ALITE_merge':  compute_summary(df_alite),
    'Manual_merge': compute_summary(df_manual)
}).T

# — 2. Country–Year Alignment (Jaccard) —
if cc and yy:
    set_a  = set(df_alite [[cc, yy]].drop_duplicates().apply(tuple, axis=1))
    set_m  = set(df_manual[[cc, yy]].drop_duplicates().apply(tuple, axis=1))
    inter  = len(set_a & set_m)
    uni    = len(set_a | set_m)
    jacc   = inter / uni if uni else np.nan
    align_df = pd.DataFrame([{
        'Shared Country-Years': inter,
        'Total Country-Years':  uni,
        'Jaccard Index':        round(jacc, 3)
    }])
else:
    align_df = pd.DataFrame()

# — 3. Coverage of Unique Countries & Years —
if cc and yy:
    countries_a = set(df_alite [cc].dropna())
    countries_m = set(df_manual[cc].dropna())
    years_a     = set(df_alite [yy].dropna())
    years_m     = set(df_manual[yy].dropna())
    coverage_df = pd.DataFrame({
        'ALITE_merge':  [len(countries_a), len(years_a)],
        'Manual_merge': [len(countries_m), len(years_m)],
        'Common':       [len(countries_a & countries_m),
                         len(years_a     & years_m)]
    }, index=['Unique Countries','Unique Years'])
else:
    coverage_df = pd.DataFrame()

# — 4. Column Overlap & Listings —
cols_a  = set(df_alite .columns)
cols_m  = set(df_manual.columns)
common  = cols_a & cols_m
overlap_df = pd.DataFrame([{
    'ALITE_only':       len(cols_a - common),
    'Common':           len(common),
    'Manual_only':      len(cols_m - common),
    'ALITE_only_cols':  ', '.join(sorted(cols_a - common)),
    'Manual_only_cols': ', '.join(sorted(cols_m - common))
}])

# — 5. Top-5 Missingness in Each —
missing_a = (
    df_alite.isna().mean() * 100
).sort_values(ascending=False).head(5).round(2).reset_index()
missing_a.columns = ['Column','Missing (%)']

missing_m = (
    df_manual.isna().mean() * 100
).sort_values(ascending=False).head(5).round(2).reset_index()
missing_m.columns = ['Column','Missing (%)']

# — 6. Data-Type Consistency Counts —
dtype_records = []
for col in sorted(common):
    a_t = df_alite[col].dtype
    m_t = df_manual[col].dtype
    dtype_records.append({
        'Column':       col,
        'ALITE_dtype':  str(a_t),
        'Manual_dtype': str(m_t),
        'Consistent':   a_t == m_t
    })
dtype_df = pd.DataFrame(dtype_records)
dtype_summary_df = pd.DataFrame([{
    'Consistent':   int(dtype_df['Consistent'].sum()),
    'Inconsistent': int((~dtype_df['Consistent']).sum())
}])

# — 7. Numeric Comparison (mean, MAD, KS-test) —
numeric_common = [
    col for col in common
    if pd.api.types.is_numeric_dtype(df_alite[col])
    and pd.api.types.is_numeric_dtype(df_manual[col])
    and col not in {cc, yy}
]

# if keys exist, build merged for MAD
if cc and yy:
    merged = pd.merge(
        df_alite [[cc, yy] + numeric_common],
        df_manual[[cc, yy] + numeric_common],
        on=[cc, yy],
        suffixes=('_alite','_manual')
    )

stats = []
for col in numeric_common:
    a_vals = df_alite [col].dropna()
    m_vals = df_manual[col].dropna()
    if len(a_vals) > 1000: a_vals = a_vals.sample(1000, random_state=1)
    if len(m_vals) > 1000: m_vals = m_vals.sample(1000, random_state=1)
    if not a_vals.empty and not m_vals.empty:
        ks_s, p_v = ks_2samp(a_vals, m_vals)
        mad = np.nan
        if cc and yy:
            diff = merged[f'{col}_alite'] - merged[f'{col}_manual']
            mad  = diff.abs().mean()
        stats.append({
            'Column':      col,
            'ALITE_mean':  round(a_vals.mean(), 3),
            'Manual_mean': round(m_vals.mean(), 3),
            'MeanAbsDiff': round(mad, 3),
            'KS_stat':     round(ks_s, 3),
            'KS_pval':     '{:.3e}'.format(p_v)
        })

stats_df = pd.DataFrame(stats).sort_values(by='MeanAbsDiff', ascending=False)

# — 8. Print Everything —
print("\n=== Summary Metrics ===")
print(summary_df)

print("\n=== Country–Year Alignment ===")
print(align_df)

print("\n=== Coverage of Countries & Years ===")
print(coverage_df)

print("\n=== Column Overlap & Listings ===")
print(overlap_df.T)

print("\n=== Top-5 Missingness in ALITE_merge ===")
print(missing_a)

print("\n=== Top-5 Missingness in Manual_merge ===")
print(missing_m)

print("\n=== Data-Type Consistency Counts ===")
print(dtype_summary_df)

print("\n=== Numeric Columns Statistical Comparison ===")
print(stats_df.to_string(index=False))



=== Summary Metrics ===
                 Rows  Columns  AvgMissing (%)  UniqueCountryYear
ALITE_merge   10068.0     20.0            5.26            10068.0
Manual_merge   9142.0     20.0            2.29             9142.0

=== Country–Year Alignment ===
   Shared Country-Years  Total Country-Years  Jaccard Index
0                  8806                10404          0.846

=== Coverage of Countries & Years ===
                  ALITE_merge  Manual_merge  Common
Unique Countries          170           155     149
Unique Years               60            60      60

=== Column Overlap & Listings ===
                   0
ALITE_only         0
Common            20
Manual_only        0
ALITE_only_cols     
Manual_only_cols    

=== Top-5 Missingness in ALITE_merge ===
             Column  Missing (%)
0   effective_labor        47.11
1     total_patents        36.40
2  energy_per_labor        21.72
3           country         0.00
4              year         0.00

=== Top-5 Missingness in Man